# Gas Station Simulator Extension: Requesting Multiple Resources

#### _Edward Krueger, edkrueger@gmail.com_

## Purpose

## Overview


## The simulation details

### Customer arrivals

Alice's experiences have told her a customer arrives about every 2 minutes. However, she would like to be able to change this value in the simulation to add robustness.

You should use the exponential distribution parameterized so that the mean waiting time is 2. The exponential distribution is commonly used for waiting time between events. Its a nice, simple distribution to use because it has a single parameter distribution that is directly related to its mean.

For more, see: https://en.wikipedia.org/wiki/Exponential_distribution#Applications_of_exponential_distribution.

The scipy package implements random draws from this distribution: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.expon.html.

Be careful, because scipy's parametrization is different from the one in the Wikipedia article above!


### Customer requirements

Customers, obviously, utilize pumps when they need gas. We make the assumption that a customer's gas requirement is normally distributed, centered around 15 gallons, with standard deviation 5. This roughly corresponds to 95% of customers requiring between 5 and 25 gallons of gas. Almost all customers will then take between 0 and 30 gallons of gas. If you like, verify this both numerically and mathematically. (Hint: look up the "empirical rule")

Many statistical packages only provide a standard normal distribution with mean 0 and standard deviation 1. But, by multiplying by the standard deviation and adding the mean you can transform the random variable.

Mathematically, if $z \sim N(0,1)$ and $x \sim N(\mu, \sigma)$ then $z * \sigma + \mu = x$. Note, this formula is the inverse of the formula for standardizing a normal random variable.

Scipy has the normal distribution available in the same module as the exponential distribution: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html.

To be safe, since the normal distribution has support of $(-\infty, \infty)$, you'll want to make sure that the draw is always nonnegative.

### Pump rate

Many pumps can pump at a rate faster than 10 gallons per minute, the US law mandates a cap of 10 gallons per minute. Alice's pumps are this fast.

### Customer behavior

Some customers just pump gas, but most have some lay time. They go in to buy snacks, pay for gas with cash, go to the bathroom, etc.

You'll model lay time with an exponential distri
bution with mean 5.

Customer's will spend the higher of their pump time and their lay time utilizing the pump and the leave allowing the next costumer to use the pump.

## Implementation

I would sugest using the package simpy to write your solution, as that is the package I've written mine in. However, there are many other options in many languages. This is simple enoguh that you could even write this from scratch; but I wouldn't advise it. Simpy has a good, quick, 10-minute tutorial at https://simpy.readthedocs.io/en/latest/simpy_intro/index.html#intro.

Copyright © 2018 Edward Krueger

The problem is that we think there are enough cars that black both pumps. We'll simplify the simulation so that there are two cars and cars are assigned a pump when they arrive at the gas station.

# Gas Station Simulator Solution

## Setup

In [1]:
import simpy

import numpy as np

from scipy.stats import expon
from scipy.stats import norm

from contextlib import ExitStack

In [2]:
class GasPump():
    
    """
    Models a gas pump as a simpy.Resource with a given number of pumps.
    
    Arguments:
    -----------
    num_pumps (int): The number of pumps at the gas station
    env (simpy.Environment): The simpy environment
    """
    
    def __init__(self, name, env):
        
        self.name = name
                
        self.env = env
        
        self.resource = simpy.Resource(self.env, capacity=1)
        
        self.minutes_utilized = 0
        self.gallons_sold = 0

In [3]:
class Car():

    """
    Models a car.
    The car's simulation process in the simulation is the method .run().
    
    Arguments:
    -----------
    id (int): A unique id for the car
    gas_required (float): Amount of gallons a car will fill
    lay_time (float): Extra minutes that the car stays at the pump
    gas_pump (GasStation): The gas pumps the car is assigned to.
    env (simpy.Environment): The simpy environment
    
    """
    
    def __init__(
        self,
        id,
        gas_required,
        lay_time,
        gas_pumps,
        env
    ):
        
        self.id = id
        self.gas_required = gas_required
        self.lay_time = lay_time
        self.gas_pumps = gas_pumps
        self.env = env
        
        self.wait_time = np.inf
        self.finished = False
        
        self.pump_names = " ".join([gas_pump.name for gas_pump in gas_pumps])
        
        self.action = self.env.process(self.run())
        
    def run(self):
        
        queue_time = self.env.now
        
        print(f'Car {self.id} arrives at {round(queue_time, 2)} minutes')
        
                
        with ExitStack() as stack:
            
            reqs = [stack.enter_context(res.resource.request()) for res in self.gas_pumps]
            
            yield self.env.all_of(reqs)
            
            pump_start_time = self.env.now
            
            print(f'Car {self.id} begins utilizing pumps {self.pump_names} at {round(pump_start_time, 2)} minutes')

            pump_time = self.gas_required / PUMP_RATE
            utilization = max(pump_time, self.lay_time)
            
            yield self.env.timeout(utilization)
            
            pump_end_time = self.env.now
            
            print(f'Car {self.id} endutilizing pumps {self.pump_names} at {round(pump_end_time, 2)} minutes')
            
            self.finished = True
            
            self.wait_time = pump_start_time - queue_time
            
            # add the utilization to all pumps
            for gas_pump in self.gas_pumps:
                gas_pump.minutes_utilized += utilization
                
            # add the gas to only the first pump
            self.gas_pumps[0].gallons_sold += self.gas_required

In [4]:
def scheduler(env):
    
    """
    Controls the arrival of cars at the gas station.
    
    Arguments:
    -----------
    env (simpy.Environment): The simpy environment
    """
    
    id = 0
    
    while True:
        
        waiting_time = expon.rvs(loc=0, scale=EXPECTED_WAIT)
        
        std_norm = norm.rvs()
        gas_required = std_norm * GAS_REQUIRED_STD + GAS_REQUIRED_MEAN
        gas_required = max([0, gas_required])
        
        lay_time = expon.rvs(loc=0, scale=EXPECTED_LAY_TIME)
        
        yield env.timeout(waiting_time)
        
        gas_pumps = np.random.choice(
            a = [
                [np.random.choice(pumps)],
                pumps
            ],
            p = [
                1- PROP_LARGE_CARS,
                PROP_LARGE_CARS,
            ]
        )
        
        CARS.append(
            Car(
                id=id,
                gas_required=gas_required,
                lay_time=lay_time,
                gas_pumps=gas_pumps,
                env=env
            )
        )
        
        id += 1

## Input parameters and run the simulation

In [5]:
EXPECTED_WAIT = 2 # in minutes

GAS_REQUIRED_MEAN = 15 # in gallons
GAS_REQUIRED_STD = 5 # in gallons

PUMP_RATE = 10 # in gallons per minute

EXPECTED_LAY_TIME = 5 # in minutes

PROP_LARGE_CARS = .15

sim_time = 24 * 60 # in minutes

## Wrap up the simulation and metrics

In [6]:
env = simpy.Environment()
CARS = []

pump_1 = GasPump("1", env)
pump_2 = GasPump("2", env)
pumps = [pump_1, pump_2]

env.process(scheduler(env))
env.run(until=sim_time)


Car 0 arrives at 3.96 minutes
Car 0 begins utilizing pumps 1 at 3.96 minutes
Car 1 arrives at 7.38 minutes
Car 1 begins utilizing pumps 2 at 7.38 minutes
Car 1 endutilizing pumps 2 at 8.31 minutes
Car 2 arrives at 8.91 minutes
Car 3 arrives at 12.6 minutes
Car 4 arrives at 13.03 minutes
Car 5 arrives at 17.28 minutes
Car 6 arrives at 17.29 minutes
Car 6 begins utilizing pumps 2 at 17.29 minutes
Car 7 arrives at 18.25 minutes
Car 8 arrives at 19.55 minutes
Car 6 endutilizing pumps 2 at 21.47 minutes
Car 7 begins utilizing pumps 2 at 21.47 minutes
Car 0 endutilizing pumps 1 at 23.35 minutes
Car 2 begins utilizing pumps 1 at 23.35 minutes
Car 9 arrives at 23.73 minutes
Car 10 arrives at 25.77 minutes
Car 11 arrives at 27.46 minutes
Car 12 arrives at 28.73 minutes
Car 13 arrives at 31.6 minutes
Car 7 endutilizing pumps 2 at 32.35 minutes
Car 9 begins utilizing pumps 2 at 32.35 minutes
Car 2 endutilizing pumps 1 at 32.76 minutes
Car 3 begins utilizing pumps 1 at 32.76 minutes
Car 14 arrives

Car 460 arrives at 1033.78 minutes
Car 461 arrives at 1035.01 minutes
Car 249 endutilizing pumps 2 at 1035.06 minutes
Car 250 begins utilizing pumps 2 at 1035.06 minutes
Car 462 arrives at 1035.54 minutes
Car 248 endutilizing pumps 1 at 1035.85 minutes
Car 252 begins utilizing pumps 1 at 1035.85 minutes
Car 463 arrives at 1038.92 minutes
Car 464 arrives at 1039.26 minutes
Car 465 arrives at 1041.64 minutes
Car 466 arrives at 1042.06 minutes
Car 467 arrives at 1043.71 minutes
Car 468 arrives at 1044.91 minutes
Car 469 arrives at 1045.26 minutes
Car 470 arrives at 1045.77 minutes
Car 250 endutilizing pumps 2 at 1045.77 minutes
Car 251 begins utilizing pumps 2 at 1045.77 minutes
Car 471 arrives at 1048.62 minutes
Car 472 arrives at 1049.01 minutes
Car 473 arrives at 1049.14 minutes
Car 474 arrives at 1049.41 minutes
Car 475 arrives at 1050.03 minutes
Car 251 endutilizing pumps 2 at 1050.86 minutes
Car 253 begins utilizing pumps 2 at 1050.86 minutes
Car 476 arrives at 1051.91 minutes
Car 4

In [7]:
avg_wait_time = np.array([car.wait_time for car in CARS if car.finished]).mean()
total_gallons_sold = sum([pump.gallons_sold for pump in pumps])

for pump in pumps:
    utilization = pump.minutes_utilized / sim_time * len(pumps) 
    print(f'Utilization for pump {pump.name}: {utilization}')
    
for pump in pumps:
    gallons = pump.gallons_sold
    print(f'Gallons sold at pump {pump.name}: {gallons}')
    


print(f'Average wait time (minutes): {avg_wait_time}')
print(f'Total Gallons Sold: {total_gallons_sold}')

Utilization for pump 1: 1.4099887063411365
Utilization for pump 2: 1.7457054384700763
Gallons sold at pump 1: 2687.1901191475035
Gallons sold at pump 2: 2447.742574077391
Average wait time (minutes): 332.1970570345712
Total Gallons Sold: 5134.932693224895
